In [1]:
from text_processing import *
from constants import *

In [2]:
file_selector = is_2018_in_file

In [3]:
judgements_files = filter(file_selector, os.listdir(json_data_dir))
texts = chain.from_iterable(map(lambda f: judgements_raw(f,json_data_dir), judgements_files))
texts = islice(texts,3000)
filtered = filter(filter_judgements, texts)

with_categories = map(lambda j: WithCategory(j, map_category(j).label), filtered)
cleaned = map(
    lambda j: Line(j.judgement['id'], clean_text(j.judgement['textContent'], common_words), j.category),
    with_categories)
cleaned  = list(cleaned)


In [4]:
tuples = map(lambda l: (l.words,l.category),cleaned)
listed = list(filter(lambda tup: tup[1] in ['civil', 'criminal', 'economic', 'insurance'],tuples))

x_train, x_test, y_train, y_test = get_train_test_data(listed)

In [5]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words = common_words)),
    ('clf', OneVsRestClassifier(LinearSVC())),
])

pipeline.fit(x_train, y_train)
predictions = pipeline.predict(y_test)

print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

      civil       0.00      0.00      0.00        48
   criminal       0.19      1.00      0.32        30
   economic       0.00      0.00      0.00        24
  insurance       0.00      0.00      0.00        56

avg / total       0.04      0.19      0.06       158



/home/marcin/tens1-6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [6]:
cat_id = dict(map(lambda line: (line.id, line.category), cleaned))
tagged_categories = list(tagged(cat_id))